In [75]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeCV
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate, train_test_split
import mat4py
from sklearn import metrics
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
import torch
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import mat4py
import pandas as pd
import data_tracks
import models
import pickle as pkl
from style import *
import viz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
df = data_tracks.get_data()

[-232.38214757977647,
 -139.55371689069582,
 6.529149404742989,
 -172.85467842444706,
 -42.6024665436692,
 -65.60005615567643,
 -50.535679149008665,
 385.9307737628889,
 -88.4889545038215,
 53.12593761591304,
 70.52356201417251,
 -115.8331526539381,
 852.1186330216076,
 -119.21862692168824,
 29.78411803187516,
 -26.427885815577795,
 -207.52943086248942,
 -22.73665247406564,
 37.25491230348702,
 -27.9430125467901,
 -8.736211376791687,
 181.2885492473019,
 45.09683603146709,
 -33.13397633688172,
 -145.20299448843383,
 -95.73104634782177,
 431.738767537399,
 24.054982634794573,
 -68.78723179205008,
 -187.2335094003566,
 -0.6689469218476249,
 165.8344410684973,
 11.903349639578234,
 112.71842617651365,
 -33.84776351380453,
 2.79120302404936,
 -144.41749598219377,
 64.51930422681939,
 -88.57547739346988,
 -120.11763291083057,
 149.80665001882298,
 146.69168292056835,
 55.62368235571739,
 12.179210155941194,
 -132.86996319909562,
 -98.90590091076436,
 -104.08273171808521,
 -101.0037218813397

# look at tracks themselves

In [4]:
auxilin_dir = '/scratch/users/vision/data/abc_data/auxilin_data_tracked'

    # 8 cell folders [1, 2, 3, ..., 8]
cell_num = 1
fname = f'{auxilin_dir}/A7D2/Cell{cell_num}_1s/TagRFP/Tracking/ProcessedTracks.mat'
# cla, aux = get_images(f'Cell{cell_num}_1s', auxilin_dir=auxilin_dir)
# fname_image = data_dir = oj(auxilin_dir, 'A7D2', f'Cell{cell_num}_1s')
mat = mat4py.loadmat(fname)
tracks = mat['tracks']

In [5]:
tracks.keys()

dict_keys(['t', 'f', 'x', 'y', 'A', 'c', 'x_pstd', 'y_pstd', 'A_pstd', 'c_pstd', 'sigma_r', 'SE_sigma_r', 'pval_Ar', 'isPSF', 'tracksFeatIndxCG', 'gapVect', 'gapStatus', 'gapIdx', 'seqOfEvents', 'nSeg', 'visibility', 'lifetime_s', 'start', 'end', 'startBuffer', 'endBuffer', 'MotionAnalysis', 'maskA', 'maskN', 'RSS', 'mask_Ar', 'hval_Ar', 'hval_AD', 'catIdx', 'isCCP', 'significantMaster', 'significantVsBackground', 'significantSlave'])

# visualize a single example

In [ ]:
# pick a single interesting example
# ex = df[df.lifetime > 250].iloc[0]
ex = df[(df.lifetime > 30) * (df.y_consec_sig == 1)]
ex = ex.sort_values(by='Y_max')
# .iloc[5]
ex = ex.iloc[-5]
plt.figure(dpi=200)
plt.plot(ex['X'], color='red', label='clathrin')
plt.plot(ex['Y'], color='green', label='auxilin')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

# basic eda

In [ ]:
viz.plot_curves(df, hline=False)

In [ ]:
plt.plot(df['x_pos'], df['y_pos'], 'o', alpha=0.1)

# look at learned dictionaries

In [ ]:
print(os.listdir('processed/dictionaries'))
# d = pkl.load(open(f'processed/dictionaries/' + 'sc_12_alpha=1.pkl', 'rb'))
d = pkl.load(open(f'processed/dictionaries/' + 'nmf_12.pkl', 'rb'))
R, C = 3, 4
i = 0
plt.figure(dpi=200)
vmin = d.components_.min()
vmax = d.components_.max()
# print('err', d.reconstruction_err_ / np.linalg.norm(X_mat, ord='fro'))
for r in range(R):
    for c in range(C):
        plt.subplot(R, C, i + 1)
        plt.plot(d.components_[i], color=cr)
        i += 1
        plt.ylim((vmin, vmax))
        if r < R-1:
            plt.xticks([])
        if c > 0:
            plt.yticks([])
plt.tight_layout()
plt.show()

# viz tracks where auxilin peaks first

In [ ]:
outcomes = ['y_thresh', 'y_consec_sig', 'y_consec_thresh']
for outcome in outcomes:
    X_peak_idx = np.array([np.argmax(x) for x in df.X])
    Y_peak_idx = np.array([np.argmax(y) for y in df.Y])
    idxs_err = Y_peak_idx < X_peak_idx
    df_err = df[idxs_err]
    df_err_pos = df_err[df_err[outcome] == 1]
    num_pos = df[df[outcome] == 1].shape[0]
    num_pos_err = df_err_pos.shape[0] 
    frac_pos_err = num_pos_err / num_pos
    print('num tot', df.shape[0], 'num aux peaks first', df_err.shape[0], f'frac pos aux peaks first {frac_pos_err:0.3f}')
    viz.plot_curves(df_err_pos)

In [ ]:
plt.figure(dpi=300)
plt.hist(df_err_pos.lifetime, bins=20, color=cb)
plt.xlabel('lifetime')
plt.ylabel('count')
plt.title('aux+ with auxilin peaking first')
plt.show()

In [ ]:
R, C = 2, 1
plt.figure(dpi=200)
plt.subplot(R, C, 1)
plt.hist(X_peak_idx[idxs_err], color=cr)
plt.xlabel('Clathrin peak time')
plt.ylabel('Count')
plt.subplot(R, C, 2)
plt.hist(Y_peak_idx[idxs_err], color=cr)
plt.xlabel('Auxilin peak time')
plt.ylabel('Count')
plt.tight_layout()

# visualize splines

In [ ]:
viz.plot_curves(df_err, extra_key='X_smooth_spl')

# analyze tracking

In [1]:
fname = f'/scratch/users/vision/data/abc_data/auxilin_data_tracked/A7D2/Cell1_1s/TagRFP/Tracking/ProcessedTracks.mat'
mat = mat4py.loadmat(fname)
tracks = mat['tracks']

In [3]:
i = 2
x = tracks['x'][i][0]
xx = tracks['x'][i][1]

print('clath', len(x), 'aux', len(xx))

clath 300 aux 300


In [ ]:
for i in range(1000):
    print(tracks['t'][i][:5])

# new feats
**gets the clathrin traces aligned by their maximum, padded by zeros when necessary**

In [75]:
def get_peak(row, num=21):
    vals = np.empty(2 * num + 1) * np.nan 
    # vals = np.zeros(2 * num + 1)
    x = row['X']
    peak_idx = row['X_peak_idx']
    x_before = x[max(0,  peak_idx - num): peak_idx]
    
    vals[num - len(x_before): num] = x_before
    vals[num] = x[peak_idx]
    
    x_after = x[peak_idx + 1: peak_idx + num + 1]
    vals[num + 1: num + 1 + len(x_after)] = x_after
    return vals
mat = np.vstack(df.apply(get_peak, axis=1).values)
window_size = mat[0].size
mid = (window_size // 2)

In [ ]:
outcome_def = 'y_consec_thresh'
R, C = 8, 8
i = 0
plt.figure(figsize=(C, R), dpi=200)
vmin = np.nanmin(mat)
vmax = np.nanmax(mat)
# print('err', d.reconstruction_err_ / np.linalg.norm(X_mat, ord='fro'))

pos = df[outcome_def]==1
args = np.argsort(df[pos]['fall_local_11'].values)
mat_pos = mat[pos][args]
falls = df['fall_local_11'][pos].iloc[args].values

for r in range(R):
    for c in range(C):
        plt.subplot(R, C, i + 1)
        plt.title(f'{falls[i]:.2e}')
        plt.plot(mat_pos[i * 8], color=cb, lw=1)
        plt.plot(mid, mat_pos[i * 8][mid], 'o', color='white', ms=2)
        
        
        plt.ylim((vmin, vmax))
        if r < R-1:
            plt.xticks([])
        if c > 0:
            plt.yticks([])
            
        i += 1
plt.tight_layout()
# plt.savefig('peak_aligned.pdf')
plt.show()

# pairplot of most relevant feats

In [ ]:
d = df[['fall', 'X_max', 'rise', 'X_mean', 'y_consec_sig']]
sns.pairplot(d, hue='y_consec_sig')
plt.show()

In [ ]:
d = df[['fall', 'fall_extended', 'fall_late_extended'] + ['y_consec_sig']]
sns.pairplot(d, hue='y_consec_sig')
plt.show()

# spike-times plot

In [ ]:
plt.figure(dpi=300)
plt.hist(df['Y_peak_idx'] - df['X_peak_idx'], bins=20)
plt.xlabel('aux peak time - clath peak time')
plt.ylabel('count')
plt.show()

# spike magnitude plot

In [ ]:
plt.figure(dpi=300)
plt.scatter(df['X_max'], df['fall'], c=df['Y_max'], alpha=0.5, marker='.')
plt.colorbar()
plt.xlabel('clath max')
plt.ylabel('fall')
plt.show()